# Build and Optimize Data Warehouses with BigQuery

In [1]:
%%bash

python3 -m pip install -q -U pip setuptools wheel
python3 -m pip install -q -U numpy pandas matplotlib seaborn pyarrow tqdm
python3 -m pip install -q -U google-cloud-bigquery google-cloud-bigquery-storage

In [2]:
import os
from google.cloud import bigquery

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/meng/work/.GCP_SA/mlee-claritas-bigdata-poc.json"

In [3]:
%load_ext google.cloud.bigquery

## BigQuery command-line

In [ ]:
bq query --use_legacy_sql=false \
"""SELECT
   word,
   SUM(word_count) AS count
 FROM
   `bigquery-public-data.samples.shakespeare`
 WHERE
   word LIKE "%raisin%"
 GROUP BY
   word"""

In [ ]:
# Create a new dataset
bq mk babynames

# Create table
bq load babynames.names2010 yob2010.txt \
name:string,gender:string,count:integer

# Remove dataset
bq rm -r babynames

## Joins & Unions

In [ ]:
# calculate ratio and filter
SELECT DISTINCT
  website.productSKU,
  website.total_ordered,
  inventory.name,
  inventory.stockLevel,
  inventory.restockingLeadTime,
  inventory.sentimentScore,
  inventory.sentimentMagnitude,
  SAFE_DIVIDE(website.total_ordered, inventory.stockLevel) AS ratio
FROM
  ecommerce.sales_by_sku_20170801 AS website
  LEFT JOIN `data-to-insights.ecommerce.products` AS inventory
  ON website.productSKU = inventory.SKU
# gone through more than 50% of inventory for the month
WHERE SAFE_DIVIDE(website.total_ordered,inventory.stockLevel) >= .50
ORDER BY total_ordered DESC

In [ ]:
# Insert the sales record
INSERT INTO ecommerce.sales_by_sku_20170802
(productSKU, total_ordered)
VALUES('GGOEGHPA002910', 101)

The difference between a `UNION` and `UNION ALL` is that a `UNION` will not include duplicate records.

In [ ]:
SELECT * FROM ecommerce.sales_by_sku_20170801
UNION ALL
SELECT * FROM ecommerce.sales_by_sku_20170802

In [ ]:
# Use the table wildcard filter and _TABLE_SUFFIX filter
SELECT * FROM `ecommerce.sales_by_sku_2017*`
WHERE _TABLE_SUFFIX = '0802'

## Date-Partitioned Tables

In [ ]:
# Create a new partitioned table based on date
CREATE OR REPLACE TABLE ecommerce.partition_by_day
PARTITION BY date_formatted
OPTIONS(
  description="a table partitioned by date"
) AS
SELECT DISTINCT
  PARSE_DATE("%Y%m%d", date) AS date_formatted,
  fullvisitorId
FROM `data-to-insights.ecommerce.all_sessions_raw`

In [ ]:
# The table wildcard * used in the `FROM` clause to limit the amount of tables 
# referred to in the `TABLE_SUFFIX` filter.
# partition_expiration_days = 60
 CREATE OR REPLACE TABLE ecommerce.days_with_rain
 PARTITION BY date
 OPTIONS (
   partition_expiration_days=60,
   description="weather stations with precipitation, partitioned by day"
 ) AS
 SELECT
   DATE(CAST(year AS INT64), CAST(mo AS INT64), CAST(da AS INT64)) AS date,
   (SELECT ANY_VALUE(name) FROM `bigquery-public-data.noaa_gsod.stations` AS stations
    WHERE stations.usaf = stn) AS station_name,  -- Stations may have multiple names
   prcp
 FROM `bigquery-public-data.noaa_gsod.gsod*` AS weather
 WHERE prcp < 99.9  -- Filter unknown values
   AND prcp > 0      -- Filter
   AND CAST(_TABLE_SUFFIX AS int64) >= 2018

## Join Pitfalls

In [8]:
%%bigquery
# aggregate all the product SKUs that are associated with one product name
SELECT
  v2ProductName,
  COUNT(DISTINCT productSKU) AS SKU_count,
  STRING_AGG(DISTINCT productSKU LIMIT 5) AS SKU
FROM `data-to-insights.ecommerce.all_sessions_raw`
WHERE productSKU IS NOT NULL
GROUP BY v2ProductName
HAVING SKU_count > 1
ORDER BY SKU_count DESC
LIMIT 10;

Downloading: 100%|██████████| 10/10 [00:03<00:00,  2.87rows/s]


,v2ProductName,SKU_count,SKU
0,Waze Women's Typography Short Sleeve Tee,12,"9184709,GGOEWXXX0834,9184711,9184710,9184705"
1,Google Women's Insulated Thermal Vest Navy,10,"9182760,GGOEGAAX0585,GGOEGAPL058513,GGOEGAPL05..."
2,Android Women's Short Sleeve Badge Tee Dark He...,10,"GGOEGAAX0282,GGOEAAEJ028213,9182176,GGOEAAEJ02..."
3,Google Men's Watershed Full Zip Hoodie Grey,10,"GGOEGAAX0568,9182739,9182383,9182382,GGOEGADJ0..."
4,Google Sunglasses,10,"GGOEGAAX0037,9180829,GGOEGHGR019499,GGOEGHGH01..."
5,Google Men's Performance Full Zip Jacket Black,9,"GGOEGAAX0569,9182751,9184620,GGOEGADB056915,GG..."
6,Google Women's Yoga Jacket Black,9,"9182502,GGOEGAAX0586,9182761,GGOEGAPB058614,GG..."
7,Google Men's Vintage Badge Tee White,9,"GGOEGAAX0339,9182525,GGOEGAAQ033914,GGOEGAAQ03..."
8,Android Women's Short Sleeve Hero Tee Black,9,"GGOEGAAX0283,GGOEAAEB028314,GGOEAAEB028313,GGO..."
9,Google Women's Short Sleeve Hero Tee Sky Blue,8,"GGOEGAAX0291,9182658,9184720,GGOEGAEC029114,GG..."


In [ ]:
# Unintentional many-to-one SKU relationship
WITH inventory_per_sku AS (
  SELECT DISTINCT
    website.v2ProductName,
    website.productSKU,
    inventory.stockLevel
  FROM `data-to-insights.ecommerce.all_sessions_raw` AS website
  JOIN `data-to-insights.ecommerce.products` AS inventory
    ON website.productSKU = inventory.SKU
    WHERE productSKU = 'GGOEGPJC019099'
)
SELECT
  productSKU,
  SUM(stockLevel) AS total_inventory
FROM inventory_per_sku
GROUP BY productSKU

In [10]:
%%bigquery
# Use distinct SKUs before joining
SELECT DISTINCT
website.productSKU AS website_SKU,
inventory.SKU AS inventory_SKU
FROM `data-to-insights.ecommerce.all_sessions_raw` AS website
FULL JOIN `data-to-insights.ecommerce.products` AS inventory
ON website.productSKU = inventory.SKU
WHERE website.productSKU IS NOT NULL OR inventory.SKU IS NOT NULL
LIMIT 10;

Downloading: 100%|██████████| 10/10 [00:03<00:00,  2.88rows/s]


,website_SKU,inventory_SKU
0,9180874,None
1,9182713,None
2,9182784,9182784
3,GGOEGAFB035816,GGOEGAFB035816
4,9180803,None
5,9183074,None
6,GGOENEBB081499,GGOENEBB081499
7,GGOEYAAJ033015,GGOEYAAJ033015
8,GGOEGAAJ073017,None
9,GGOEGAEB028617,None


In [ ]:
# Unintentional Cross Join
SELECT DISTINCT
productSKU,
v2ProductCategory,
discount
FROM `data-to-insights.ecommerce.all_sessions_raw` AS website
CROSS JOIN ecommerce.site_wide_promotion
WHERE v2ProductCategory LIKE '%Clearance%'
AND productSKU = 'GGOEGOLC013299'
LIMIT 10;

## Working with JSON, Arrays, and Structs in BigQuery

### Arrays

In [13]:
%%bigquery
SELECT
  fullVisitorId,
  date,
  ARRAY_AGG(v2ProductName) AS products_viewed,
  ARRAY_AGG(pageTitle) AS pages_viewed
  FROM `data-to-insights.ecommerce.all_sessions`
WHERE visitId = 1501570398
GROUP BY fullVisitorId, date
ORDER BY date

Downloading: 100%|██████████| 2/2 [00:03<00:00,  1.74s/rows]


,fullVisitorId,date,products_viewed,pages_viewed
0,5710379250208908569,20170731,[Google Women's Lightweight Microfleece Jacket...,"[Google Snapback Hat Black, Google RFID Journal]"
1,5710379250208908569,20170801,"[Android Luggage Tag, 8 pc Android Sticker She...","[Shop by Brand | Google Merchandise Store, Off..."


In [14]:
%%bigquery
SELECT
  fullVisitorId,
  date,
  ARRAY_AGG(v2ProductName) AS products_viewed,
  ARRAY_LENGTH(ARRAY_AGG(v2ProductName)) AS num_products_viewed,
  ARRAY_AGG(pageTitle) AS pages_viewed,
  ARRAY_LENGTH(ARRAY_AGG(pageTitle)) AS num_pages_viewed
  FROM `data-to-insights.ecommerce.all_sessions`
WHERE visitId = 1501570398
GROUP BY fullVisitorId, date
ORDER BY date

Downloading: 100%|██████████| 2/2 [00:03<00:00,  1.75s/rows]


,fullVisitorId,date,products_viewed,num_products_viewed,pages_viewed,num_pages_viewed
0,5710379250208908569,20170731,[Google Women's Lightweight Microfleece Jacket...,2,"[Google Snapback Hat Black, Google RFID Journal]",2
1,5710379250208908569,20170801,"[Metal Texture Roller Pen, Waze Pack of 9 Deca...",109,[Writing Instruments | Office | Google Merchan...,109


In [15]:
%%bigquery
SELECT
  fullVisitorId,
  date,
  ARRAY_AGG(DISTINCT v2ProductName) AS products_viewed,
  ARRAY_LENGTH(ARRAY_AGG(DISTINCT v2ProductName)) AS distinct_products_viewed,
  ARRAY_AGG(DISTINCT pageTitle) AS pages_viewed,
  ARRAY_LENGTH(ARRAY_AGG(DISTINCT pageTitle)) AS distinct_pages_viewed
  FROM `data-to-insights.ecommerce.all_sessions`
WHERE visitId = 1501570398
GROUP BY fullVisitorId, date
ORDER BY date

Downloading: 100%|██████████| 2/2 [00:03<00:00,  1.84s/rows]


,fullVisitorId,date,products_viewed,distinct_products_viewed,pages_viewed,distinct_pages_viewed
0,5710379250208908569,20170731,[Google Women's Lightweight Microfleece Jacket...,2,"[Google Snapback Hat Black, Google RFID Journal]",2
1,5710379250208908569,20170801,"[Android Luggage Tag, 8 pc Android Sticker She...",61,"[Shop by Brand | Google Merchandise Store, Off...",8


In [16]:
%%bigquery
SELECT DISTINCT
  visitId,
  h.page.pageTitle
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`,
UNNEST(hits) AS h
WHERE visitId = 1501570398
LIMIT 10

Downloading: 100%|██████████| 9/9 [00:02<00:00,  3.79rows/s]


,visitId,pageTitle
0,1501570398,Fun | Accessories | Google Merchandise Store
1,1501570398,Home
2,1501570398,Shop by Brand | Google Merchandise Store
3,1501570398,Office | Google Merchandise Store
4,1501570398,Other | Office | Google Merchandise Store
5,1501570398,Writing Instruments | Office | Google Merchand...
6,1501570398,Accessories | Google Merchandise Store
7,1501570398,Electronics | Google Merchandise Store
8,1501570398,Apparel | Google Merchandise Store


### STRUCTs

A separate table that is already pre-joined into main table.

A STRUCT can have:
- one or many fields in it
- the same or different data types for each field
- it's own alias

In [19]:
%%bigquery
SELECT
  visitId,
  totals.*,
  device.*
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`
WHERE visitId = 1501570398

Downloading: 100%|██████████| 1/1 [00:03<00:00,  3.27s/rows]


,visitId,visits,hits,pageviews,timeOnSite,bounces,transactions,transactionRevenue,newVisits,screenviews,...,mobileDeviceModel,mobileInputSelector,mobileDeviceInfo,mobileDeviceMarketingName,flashVersion,javaEnabled,language,screenColors,screenResolution,deviceCategory
0,1501570398,1,11,11,731,NaN,NaN,NaN,1,NaN,...,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,None,not available in demo dataset,not available in demo dataset,not available in demo dataset,tablet


In [20]:
%%bigquery
SELECT STRUCT("Rudisha" as name, [23.4, 26.3, 26.4, 26.1] as splits) AS runner

Downloading: 100%|██████████| 1/1 [00:03<00:00,  3.45s/rows]


,runner
0,"{'name': 'Rudisha', 'splits': [23.4, 26.3, 26...."


In [ ]:
SELECT race, participants.name
FROM racing.race_results
CROSS JOIN
race_results.participants # full STRUCT name

In [ ]:
SELECT race, participants.name
FROM racing.race_results AS r, r.participants

In [ ]:
SELECT COUNT(p.name) AS racer_count
FROM racing.race_results AS r, UNNEST(r.participants) AS p

In [ ]:
SELECT
  p.name,
  SUM(split_times) as total_race_time
FROM racing.race_results AS r
, UNNEST(r.participants) AS p
, UNNEST(p.splits) AS split_times
WHERE p.name LIKE 'R%'
GROUP BY p.name
ORDER BY total_race_time ASC;

In [ ]:
SELECT
  p.name,
  split_time
FROM racing.race_results AS r
, UNNEST(r.participants) AS p
, UNNEST(p.splits) AS split_time
WHERE split_time = 23.2;

## Covid-19 policy tracker

In [ ]:
# Create a table partitioned by date
CREATE OR REPLACE TABLE covid_607.oxford_policy_tracker_110
PARTITION BY date
OPTIONS (
    partition_expiration_days=90
) AS 
SELECT 
*
FROM `bigquery-public-data.covid19_govt_response.oxford_policy_tracker`
WHERE alpha_3_code NOT IN ('GBR', 'BRA', 'CAN', 'USA');

In [ ]:
# Add new columns to table
ALTER TABLE covid_607.oxford_policy_tracker_110
  ADD COLUMN population INTEGER,
  ADD COLUMN country_area FLOAT64,
  ADD COLUMN mobility STRUCT<
    avg_retail FLOAT64,
    avg_grocery FLOAT64,
    avg_parks FLOAT64,
    avg_transit FLOAT64,
    avg_workplace FLOAT64,
    avg_residential FLOAT64
  >

In [ ]:
# Add country population data to the population column
UPDATE
    covid_607.oxford_policy_tracker_110 AS t0
SET
    t0.population = t1.pop_data_2019
FROM
    (SELECT DISTINCT 
        country_territory_code, pop_data_2019 
    FROM 
        `bigquery-public-data.covid19_ecdc.covid_19_geographic_distribution_worldwide`
    ) AS t1
WHERE
    t0.alpha_3_code = t1.country_territory_code;

In [ ]:
# Add country area data to the country_area column
UPDATE
    covid_607.oxford_policy_tracker_110 t0
SET
    t0.country_area = t1.country_area
FROM
    (SELECT DISTINCT 
        country_name, country_area
    FROM 
        `bigquery-public-data.census_bureau_international.country_names_area`
    ) AS t1
WHERE 
    t0.country_name = t1.country_name;

In [ ]:
# Populate the mobility record data
UPDATE
   covid_607.oxford_policy_tracker_110 AS t0
SET
    t0.mobility.avg_retail = t1.avg_retail,
    t0.mobility.avg_grocery = t1.avg_grocery,
    t0.mobility.avg_parks = t1.avg_parks,
    t0.mobility.avg_transit = t1.avg_transit,
    t0.mobility.avg_workplace = t1.avg_workplace,
    t0.mobility.avg_residential = t1.avg_residential
FROM
    (SELECT country_region, date,
        AVG(retail_and_recreation_percent_change_from_baseline) as avg_retail,
        AVG(grocery_and_pharmacy_percent_change_from_baseline) as avg_grocery,
        AVG(parks_percent_change_from_baseline) as avg_parks,
        AVG(transit_stations_percent_change_from_baseline) as avg_transit,
        AVG(workplaces_percent_change_from_baseline) as avg_workplace,
        AVG(residential_percent_change_from_baseline) as avg_residential
    FROM 
        `bigquery-public-data.covid19_google_mobility.mobility_report`
    GROUP BY 
        country_region, date
    ) AS t1
WHERE
    t0.country_name = t1.country_region
    AND t0.date = t1.date;

In [ ]:
# Query missing data in population & country_area
SELECT DISTINCT country_name
FROM covid_458.oxford_policy_tracker_157
WHERE population is NULL

UNION ALL

SELECT DISTINCT country_name
FROM covid_458.oxford_policy_tracker_157
WHERE country_area is NULL
ORDER BY country_name ASC